# Feature Selection - Mutual Information Gain (Entropy) For Classification

## Mutual Information

In probability theory and information theory, the mutual information (MI) of two random variables is a measure of the mutual dependence between the two variables. More specifically, it quantifies the "amount of information" (in units such as shannons, commonly called bits) obtained about one random variable through observing the other random variable. The concept of mutual information is intimately linked to that of entropy of a random variable, a fundamental notion in information theory that quantifies the expected "amount of information" held in a random variable.

Download Data Files https://github.com/laxmimerit/Data-Files-for-Feature-Selection

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# VarianceThreshold - Feature selector that removes all low-variance features.
from sklearn.feature_selection import (
    SelectKBest,
    SelectPercentile,
    VarianceThreshold,
    mutual_info_classif,
    mutual_info_regression,
)
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [ ]:
data = pd.read_csv("data/santander.csv", nrows=20000)
data.head()

In [ ]:
x = data.drop("TARGET", axis=1)  # Features
y = data["TARGET"]  # Outcome

x.shape, y.shape

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=0, stratify=y
)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

### Constant Features Removal

In [ ]:
constant_filter = VarianceThreshold(threshold=0)
constant_filter.fit(x_train)

In [ ]:
# No. of features after constants removal
constant_filter.get_support().sum()

In [ ]:
# Returns True for all the features which are constants.
constant_list = [
    not temp for temp in constant_filter.get_support()
]  # Inversing the True to False and False to True
constant_list

In [ ]:
# Name of all the features which are constants
x.columns[constant_list]

In [ ]:
# removing all the constants from our Training and Test dataset.
x_train_filter = constant_filter.transform(x_train)
x_test_filter = constant_filter.transform(x_test)

In [ ]:
# Now take a look at the original and the transformed data (after removing the constants)
x_train.shape, x_test.shape, x_train_filter.shape, x_test_filter.shape

## Quasi Constants Feature Removal

In [ ]:
quasi_constant_filter = VarianceThreshold(threshold=0.01)

In [ ]:
quasi_constant_filter.fit(x_train_filter)

In [ ]:
quasi_constant_filter.get_support().sum()

In [ ]:
x_train_quasi_filter = quasi_constant_filter.transform(x_train_filter)
x_test_quasi_filter = quasi_constant_filter.transform(x_test_filter)

In [ ]:
# Now take a look at the original and the transformed data (after removing the constants)
x_train.shape, x_test.shape, x_train_filter.shape, x_test_filter.shape, x_train_quasi_filter.shape, x_train_quasi_filter.shape

## Duplicate Features Removal

In [ ]:
x_train_T = x_train_quasi_filter.T
x_test_T = x_test_quasi_filter.T

In [ ]:
# As we can see the pandas dataframe has been transformed in to numpy array after transpose.
type(x_train_T)

In [ ]:
# Changing numpy array back to pandas dataframe
x_train_T = pd.DataFrame(x_train_T)
x_test_T = pd.DataFrame(x_test_T)

In [ ]:
# Now we can see after transpose the rows has become columns and columns has become rows.
x_train_T.shape, x_test_T.shape

In [ ]:
# Getting duplicate features count
x_train_T.duplicated().sum()

In [ ]:
duplicated_features = x_train_T.duplicated()
duplicated_features

# True is duplicated and False is non duplicated rows.

In [ ]:
# Removing duppicated features.
# After this the False becomes True and True becomes false.

# Inversing the True to False and False to True
features_to_keep = [not index for index in duplicated_features]
features_to_keep

In [ ]:
# Final dataset after removing constants, quasi constants and duplicates.

# Transposing again to original form
x_train_unique = x_train_T[features_to_keep].T

# Transposing again to original form
x_test_unique = x_test_T[features_to_keep].T

In [ ]:
x_train.shape, x_test.shape, x_train_unique.shape, x_test_unique.shape

## Build Model and Compare the Performance after and before removal.

In [ ]:
def run_random_forest(x_train, x_test, y_train, y_test):
    clf = RandomForestClassifier(n_estimators=100, random_state=0, n_jobs=-1)
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    print("Accuracy on test set: ")
    print(accuracy_score(y_test, y_pred))

In [ ]:
%%time
# Run on final data.
run_random_forest(x_train_unique, x_test_unique, y_train, y_test)

In [ ]:
%%time
# Run on original data.
run_random_forest(x_train, x_test, y_train, y_test)

As we can see the accuracy and time taken is less after removing the constants, quasi constants and duplicates compare to the original data. 

What we can say here is that removing constants, quasi constants and duplicates doesn't depricates the accuracy it rather improves it.

## Calculate Mutual Information

In [ ]:
mi = mutual_info_classif(x_train_unique, y_train)

In [ ]:
len(mi)

In [ ]:
mi

In [ ]:
mi = pd.Series(mi)
mi.index = x_train_unique.columns

In [ ]:
mi.sort_values(ascending=False, inplace=True)

In [ ]:
mi.plot.bar(figsize=(16, 5))

As you can see after certain level there is not much information

In [ ]:
# As you can see after certain level there is not much information.
# Selecting only certain percentile of entire data.
sel = SelectPercentile(mutual_info_classif, percentile=10).fit(x_train_unique, y_train)

In [ ]:
x_train_unique.columns[sel.get_support()]

In [ ]:
len(x_train_unique.columns[sel.get_support()])

In [ ]:
help(sel)

In [ ]:
x_train_mi = sel.transform(x_train_unique)
x_test_mi = sel.transform(x_test_unique)

In [ ]:
x_train_mi.shape, x_test_mi.shape

In [ ]:
%%time
run_random_forest(x_train_mi, x_test_mi, y_train, y_test)

## Mutual Information Gain in Regression

In [ ]:
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [ ]:
boston = load_boston()

In [ ]:
print(boston.DESCR)

In [ ]:
x = pd.DataFrame(data=boston.data, columns=boston.feature_names)
x.head()

In [ ]:
y = boston.target

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [ ]:
mi = mutual_info_regression(x_train, y_train)
mi = pd.Series(mi)
mi.index = x_train.columns
mi.sort_values(ascending=False, inplace=True)

In [ ]:
# Now we can see which feature is giving more information
mi

In [ ]:
mi.plot.bar()

In [ ]:
sel = SelectKBest(mutual_info_regression, k=9).fit(x_train, y_train)

In [ ]:
x_train.columns[sel.get_support()]

#### Calculate r2_score, mean_squared_error and SD for full feature

In [ ]:
model = LinearRegression()
model.fit(x_train, y_train)
y_predict = model.predict(x_test)

In [ ]:
r2_score(y_test, y_predict)

In [ ]:
# Root Mean Squared Error (RMSE)
np.sqrt(mean_squared_error(y_test, y_predict))

In [ ]:
# Standard Deviation
np.std(y)

#### Calculate r2_score, mean_squared_error and SD for selected feature

In [ ]:
# Calculating for selected features
x_train_9 = sel.transform(x_train)
x_train_9.shape

In [ ]:
x_test_9 = sel.transform(x_test)
x_test_9.shape

In [ ]:
model = LinearRegression()
model.fit(x_train_9, y_train)
y_predict_9 = model.predict(x_test_9)

In [ ]:
r2_score(y_test, y_predict_9)

In [ ]:
# Root Mean Squared Error (RMSE)
np.sqrt(mean_squared_error(y_test, y_predict_9))